In [ ]:
# this script is to develop the visual outputs for our final paper
# Create new charts folder
# folder will have example video 
# extract video and audio into memory (example file should be short)
# run pratt analysis on audio
# normalize values
# predict using saved model
# chart original, robustl2s, and our prediction

In [ ]:
#package installs
#E:\Python310\python.exe -m pip install --upgrade pip
#!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile
!E:\Python310\Scripts\pip3.exe install ffmpeg av
#!E:\Python310\python.exe -m pip uninstall torch torchvision torchaudio -y
!E:\Python310\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [1]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'charts', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [ ]:
#define variables for the process
#
# p / i / s / j / h
ANALYSIS_TYPE = 'p'
PATH_ORIGMP4 = ''
PATH_ROBMP4 = ''

PATH_MODEL = ''
if ANALYSIS_TYPE == 'p':
    PATH_MODEL = ''


In [ ]:
import parselmouth
from parselmouth.praat import call
import moviepy
from moviepy.editor import *
import moviepy.editor
from pydub import AudioSegment
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import torch
import torchvision

def create_chart_data(filesuf_mp4):

    #save all the data into this dictionary
    dict_results = {}

    #filepaths
    folderpath_base = ENV_FOLDER_DATA_PROC + 'charts\\'
    filepath_mp4 = folderpath_base + filesuf_mp4
    filepath_mp3 = folderpath_base + 'temp.mp3'
    filepath_wav = folderpath_base + 'temp.wav'
    filepath_normmp4 = folderpath_base + 'temp.mp4'
    filepath_clip = folderpath_base + 'tempclip.mp4'

    #convert mp4 to mp3
    video_clip = moviepy.editor.VideoFileClip(filepath_mp4)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(filepath_mp3)
    audio_clip.close()
    video_clip.close()

    #extract wav from mp3
    mp3_clip = AudioSegment.from_mp3(filepath_mp3)
    mp3_clip.export(filepath_wav, format="wav")

    #load wav file for pratt analysis
    sound_total = parselmouth.Sound(filepath_wav)

    #get pratt values
    list_pitch = []
    list_intensity = []
    list_shimmer = []
    list_jitter = []
    list_harmonics = []

    for t in range(40):

        start_time = t * 0.1
        end_time = (t + 1) * 0.1
        sound = sound_total.extract_part(from_time=start_time, to_time=end_time)

        #iterate through the parts of the sound

        pointprocess = call(sound, "To PointProcess (periodic, cc)",75, 600)
        #dict_temp['pointprocess'] = pointprocess
        
        #https://parselmouth.readthedocs.io/_/downloads/en/stable/pdf/
        #gets the pitch , and sets the pitch floor to 75 and tge outcg max to 600
        try:
            pitch = call(sound, "To Pitch", 0.0, 75, 600)
            val_pitch = call(pitch, "Get mean", 0, 0,"Hertz")
        except:
            val_pitch = None
        
        ##-	For intensity extraction, set the pitch floor to 100Hz. Use ‘energy’ averaging method to get mean intensity.
        try:
            intensity = call(sound, "To Intensity", 100,0.01)
            val_intensity = call(intensity, "Get mean", 0, 0,"energy")
        except:
            val_intensity = None

        ##Shimmer
        # For shimmer, extract local shimmer only, and set period floor to 0.0001s, period ceiling to 0.02s, maximum period factor to 1.3, and maximum amplitude factor to 1.6.
        try:
            val_shimmer = call([sound, pointprocess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        except:
            val_shimmer = None
        
        # For jitter, extract local jitter only, and set period floor to 0.0001s, period ceiling to 0.02s, and maximum period factor to 1.3
        #Please convert from a Sound object to a PointProcess (periodic, cc) object. (#74)
        #https://github.com/drfeinberg/PraatScripts/blob/master/Measure%20Pitch%2C%20HNR%2C%20Jitter%2C%20Shimmer%2C%20and%20Formants.ipynb
        #f0min , f0max
        try:
            val_jitter = call(pointprocess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
        except:
            val_jitter = None
        
        #-	To calculate HNR (harmonics-to-noise ratio), extract harmonicity (cc) first. Set time step to 0.01, minimum pitch to 75Hz, silence threshold to 0.1, and number of periods per window to 1.0.
        try:
            harmonics = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
            val_harmonics = call(harmonics, "Get mean", 0, 0)
        except:
            val_harmonics = None

        #add to relevant lists
        list_pitch.append(val_pitch)
        list_intensity.append(val_intensity)
        list_shimmer.append(val_shimmer)
        list_jitter.append(val_jitter)
        list_harmonics.append(val_harmonics)
    
    dict_results['p'] = list_pitch
    dict_results['i'] = list_intensity
    dict_results['s'] = list_shimmer
    dict_results['j'] = list_jitter
    dict_results['h'] = list_harmonics

    #extract target video data
    clip_total = VideoFileClip(filepath_mp4).without_audio().resize(height=360)
    clip_total = clip_total.fx(vfx.blackwhite)
    clip_total.write_videofile(filepath_normmp4, fps=10)
    #get subclip
    ffmpeg_extract_subclip(filepath_normmp4, 0, 4, targetname=filepath_clip)

    

    torchvision.set_video_backend('pyav')
    reader = torchvision.io.VideoReader(filepath_clip, "video")
    reader.set_current_stream("video")
    transform = torchvision.transforms.Grayscale()

    video_frames = torch.empty(0)
    frames = [] 
    ct = 1
    for frame in reader:
        tempframe = transform(frame['data'])
        frames.append(tempframe)
        ct += 1
    for x in range(40):
        frames.append( torch.empty((1,360,640), dtype=torch.int8) )
    if len(frames) > 0:
        video_frames = torch.stack(frames[:40], 0)
    
    dict_results['v'] = video_frames
    
    return dict_results





In [ ]:
#get the data for the original file

#use the model to create a target median + 40 data

#get the data for the robustls2

#plot 2 lines - clip median, our predicted median
#plot 3 line charts - actual, robust2lm's result, and our result


